# impute_unified_genealogy

In [ ]:
# python impute_unified_genealogy.py
# -t input.trees
# -i input vcf (sites to impute)
# -o output vcf (imputed sites)

In [ ]:
# Ancient genome imputation:
# 1. Take the unified whole-genome genealogy from Wohns et al. (2022).
# 2. Get an ancestors ts from it. (Keep leaves or not? Try both).
# 3. Get `.samples` file from input VCF.
# 4. match_samples = output ts
# 5. ts.write_vcf()
# 
# Bonus: try it on new 1KG Project data (phase 3) and evaluate using IQS.

### Downloading the unified genealogy

In [ ]:
# "[A] unified tree sequence of 3601 modern and eight high-coverage ancient human genome sequences
# compiled from eight datasets. This structure is a lossless and compact representation of
# 27 million ancestral haplotype fragments and 231 million ancestral lineages linking genomes
# from these datasets back in time.
# 
# Modern genomes:
# a. 1000 Genomes
# b. Human Genome Diversity
# c. Simons Genome Diversity
# 
# Trees:
# a. Without ancient genomes (https://zenodo.org/record/5495535#.YuGjXi8w08R)
# b. With ancient genomes (https://zenodo.org/record/5512994#.YuGj4S8w08R)
# 
# Human reference: GRCh38

In [1]:
import tskit

# Load the tree sequences of the p- and q-arms of chr20.
data_dir = "../data/ref_panel/"

contig_id = "chr20"

ts_p_file = data_dir + "hgdp_tgp_sgdp_" + contig_id + "_" + "p" + ".dated.trees"
ts_q_file = data_dir + "hgdp_tgp_sgdp_" + contig_id + "_" + "q" + ".dated.trees"

import tskit
ts_p = tskit.load(ts_p_file)
ts_q = tskit.load(ts_q_file)

print(f"The short arm of chr20 contains {ts_p.num_trees} trees.")
print(f"The long arm of chr20 contains {ts_q.num_trees} trees.")

The short arm of chr20 contains 58756 trees.
The long arm of chr20 contains 74301 trees.


In [2]:
import tsinfer

ts_p_anc = tsinfer.eval_util.make_ancestors_ts(ts_p)
ts_q_anc = tsinfer.eval_util.make_ancestors_ts(ts_q)

In [1]:
import cyvcf2
import numpy as np

In [2]:
# Simulated data from 48 high coverage shotgun (SG) samples.
# 
# Annotation file:
# http://genomebrowser-uploads.hms.harvard.edu/data/aa681/tmp/anno.sim.tsv
# 
# Simulated data files:
# http://genomebrowser-uploads.hms.harvard.edu/data/aa681/tmp/simulations.tar
# 
# MD5 checksum:
# 3a98628e5c8015b3da4664837568e14c  simulations.tar
# e332d36afc87431d08453bc66b0eee28  anno.sim.tsv
# 
# File size:
# 8.1K anno.sim.tsv
# 166G simulations.tar
# 
# There are 374 bcf files, each with 48 samples.
# I have simulated both SG and 1240k for 22 autosomes and for 8 different coverages.
# 
# */merged/{DT}/DP{DP}/chr{CHROM}.bcf
# 
# DT in [SG, 1240k]
# DP in [0.01, 0.05, 0.1, 0.2, 0.5, 1, 2, 5]
# CHROM in [1,2,...,22]
# 
# Imputed version of the original sample is also available from
#   */merged/SG/DPmax/chr{CHROM}.bcf
# 
# All 4 members of Afanasievo family are also included in this list:
#   Mother I3388.SG
#   Father I3950.SG
#   Son 1 I3949.SG
#   Son 2 I6714.SG
# 
# Please note that coverage is >20x, except for an individual from Afanasievo family with ~10x coverage (I3388.SG).  
# 
# Imputed file format is BCF with following fields:
# 
# Generated by GLIMPSE:
#   GT: Phased and imputed genotypes
#   DS: Genotype dosage
#   GP: Genotype posteriors
#
# Generated by genotype caller (mpileup):
#   PL: Phred-scaled genotype likelihoods
#   AD: Allelic depths (high-quality bases)
# 
# * Note that PL and AD are available when the target variant is covered by at least 1 read otherwise, their value is missed.
# * To minimize the reference bias, only PL of SNPs are used to build imputation model for both SNPs and indels.
#   Nevertheless, PL and AD for indels are also reported in the output.

In [3]:
data_dir = "/Users/szhan/Projects/impute_unified_genealogy/data/simulated/"
bcf_file = data_dir + "merged/SG/DPmax/" + "chr20.bcf"

In [29]:
"""
Obtain VCF with missing genotypes from a GLIMPSE imputed VCF file.
Since sequencing coverage in ancient WGS data is often too low for phasing,
the genotypes in the rebuilt VCF are all represented as haploid.

The following numbers are denoted as:
0 = REF
1 = ALT
2 = UNKNOWN

Per-sample genotypes are assigned UNKNOWN if at least one condition is met:
1. Non-SNP;
2. Total depth is 0;
3. Total depth is -1; and
4. Ref. depth is equal to alt. depth (ambiguous read support).
"""

out_vcf_file = "test.bcf.gz"
is_verbose = False

REF_GENOTYPE = (0, False,)
ALT_GENOTYPE = (1, False,)
UNK_GENOTYPE = (2, False,) # Unknown (strict_gt=True scheme)

num_sites = 0 # Total number of variable sites
num_snps = 0
num_indels = 0
num_svs = 0

vcf = cyvcf2.VCF(bcf_file, strict_gt=True)
w = cyvcf2.Writer(out_vcf_file, tmpl=vcf, mode="wb")

for v in vcf:
    # default:
    #   0=HOM_REF, 1=HET, 2=HOM_ALT, 3=UNKNOWN
    # strict_gt=True:
    #   0=HOM_REF, 1=HET, 2=UNKNOWN, 3=HOM_ALT

    num_sites += 1
    num_snps = num_snps + 1 if v.is_snp else num_snps
    num_indels = num_indels + 1 if v.is_indel else num_indels
    num_svs = num_svs + 1 if v.is_sv else num_svs

    REF_DP = v.gt_ref_depths
    ALT_DP = v.gt_alt_depths
    TOT_DP = v.gt_depths

    num_samples = v.num_called + v.num_unknown
    
    # If it is not a SNP, then assign UNKNOWN.
    # TODO: Is this appropriate?
    if not v.is_snp:
        if is_verbose:
            print(" ".join(str(x) for x in ["Non-SNP", ":", v.CHROM, v.POS, v.num_unknown, v.num_het]))
        v.genotypes = [UNK_GENOTYPE] * num_samples
        v.genotypes = v.genotypes
        continue

    # If the depths of all the samples in a site are 0 or -1, then assign UNKNOWN.
    # TODO: Ask Ali why total depth can be either 0 or -1,
    #       i.e. both ref. depth and alt. depth are either 0 or -1.
    if np.all(TOT_DP == 0) or np.all(TOT_DP) == -1:
        v.genotypes = [UNK_GENOTYPE] * num_samples
        v.genotypes = v.genotypes
        continue

    # If DP favors REF, then assign REF.
    # If DP favors ALT, then assign ALT.
    # Otherwise, assign UNKNOWN.
    is_refdp_gt_altdp = REF_DP > ALT_DP
    is_refdp_lt_altdp = REF_DP < ALT_DP
    is_refdp_et_altdp = REF_DP == ALT_DP

    is_totdp_zero = TOT_DP == 0
    is_totdp_negative_one = TOT_DP == -1
    is_unknown = is_refdp_et_altdp | is_totdp_zero | is_totdp_negative_one

    for i in np.arange(num_samples):
        if is_refdp_gt_altdp[i]:
            tmp_gt = REF_GENOTYPE
        elif is_refdp_lt_altdp[i]:
            tmp_gt = ALT_GENOTYPE
        else:
            tmp_gt = UNK_GENOTYPE
        v.genotypes[i] = tmp_gt
        
    v.genotypes = v.genotypes

    w.write_record(v)

w.close()
vcf.close()

In [22]:
print(f"{num_sites}")
print(f"{num_snps}")
print(f"{num_indels}")
print(f"{num_svs}")

1802630
1739315
63315
0
